# Introduction

In this notebook, we wil use `qwen2.5:14b` for text analysys, as well as `QWen VL` for image analysis.,

# Example of use QWen VL

A good guide of using `QWen VL` is in this [link](https://github.com/QwenLM/Qwen-VL/blob/master/TUTORIAL.md).

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig

# If you expect the results to be reproducible, set a random seed.
# torch.manual_seed(1234)

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-VL-Chat", trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-VL-Chat", device_map="cuda", trust_remote_code=True).eval()
model.generation_config = GenerationConfig.from_pretrained("Qwen/Qwen-VL-Chat", trust_remote_code=True)

The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

In [4]:
query = tokenizer.from_list_format([
    {'image': 'Rebecca_(1939_poster)_Small.jpeg'},
    {'text': 'What is the name of the movie in the poster?'},
])

In [5]:
response, history = model.chat(tokenizer, query=query, history=None)
print(response)

The name of the movie in the poster is "Rebecca."
